In [2]:
import os
import yaml


In [1]:
!pwd

/Users/taiskha/Master Thesis/code/rl_representations_main/scripts


copy stuff from train model below and mocify the original file but in a way that does not interfere with the original code

In [13]:
import os
import sys
import click
import yaml
import numpy as np
from experiment import Experiment
from pprint import pprint

import torch

# ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
# sys.path.append(ROOT_DIR)

domain = 'sepsis'
autoencoder = 'HTGNN'

def run(autoencoder, domain, options):
    dir_path = os.path.dirname(os.path.realpath(__file__))
    params = yaml.safe_load(open(os.path.join(dir_path, '../configs/common.yaml'), 'r'))    
    cfg_file = os.path.join(dir_path, '../configs/config_' + domain + f'_{autoencoder.lower()}.yaml')

    bc_config = yaml.safe_load(open(os.path.join(dir_path, '../configs/config_behavCloning.yaml'), 'r'))

    model_params = yaml.safe_load(open(cfg_file, 'r'))
    
    if autoencoder == 'CDE':
        model_params['coefs_folder'] =  os.path.join(params['storage_path'], model_params['coefs_folder'])
            
    # Iterating over the keys in model_params and replacing the values in params merging the two dictionaries
    for i in model_params:
        params[i] = model_params[i]        
    
    # Extract BC network number of nodes
    params['bc_num_nodes'] = bc_config['num_nodes']
   

    # Overriding params from config file with command line options if provided
    for opt in options:
        print(opt)
        assert opt[0] in params
        dtype = type(params[opt[0]])
        if dtype == bool:
            new_opt = False if opt[1] != 'True' else True
        else:
            new_opt = dtype(opt[1])
        params[opt[0]] = new_opt

    # Printing final parameters
    print('Parameters ')
    for key in params:
        print(key, params[key])
    print('=' * 30)

    # process param keys and values to match input to Cortex
    # what is Cortex? 
    if params['device'] == 'cuda':
        if torch.cuda.is_available():
            params['device'] = torch.device('cuda')
        else:
            params['device'] = torch.device('cpu')

    random_seed = params['random_seed']
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    random_state = np.random.RandomState(random_seed)
    # rng is not used in the code later
    # params['rng'] = random_state
    params['domain'] = domain
        
    # Update foldername to the full path 
    folder_name = params['storage_path'] + params['folder_location'] + params['folder_name']
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    params['folder_name'] = folder_name
    
    torch.set_num_threads(torch.get_num_threads())
    
    # Save model_params as a separate key in params also
    params[f'{autoencoder.lower()}_hypers'] = model_params # Cortex hyperparameter dictionaries 
    
    # Experiment
    experiment = Experiment(**params)    
    # experiment.train_autoencoder()
    experiment.train_autoencoder_gnn()
    # experiment.evaluate_trained_model()
    # experiment.train_dBCQ_policy(params['pol_learning_rate'])
    print('=' * 30)

    pprint(params)
    # with open(folder_name + '/config.yaml', 'w') as y:
    #     yaml.safe_dump(params, y)  # saving params for reference

In [ ]:
run(autoencoder, domain, [])

In [ ]:
# load each thing to the graph structure

In [ ]:
# function that gets me a graph for a timestep t
